# 문제 2: 핀다 홈화면 진입 고객의 모델 기반 고객 군집 분석/군집 별 서비스 메시지 제안

user_id 별 1행만 사용(가장 최근 행)
\
5월까지의 데이터만 사용

\
(1) **log_data** : 행동 로그 분석을 위해 전처리한 train_log_data_1.csv 사용
- app_count : 앱 사용빈도
- GetCreditInfo : 신용조회 빈도
- UseLoanManage : 대출관리 빈도
- UsePrepayCalc : 여윳돈 계산기
- UseDSRCalc : DSR 계산기 빈도
- 빈도 별 변동성(variation) : 3, 4, 5월 빈도의 회귀계수
- 신규 유저(new) : 주어진 기간 내 SignUp 이력이 있는 유저

\
(2) **user_spec + loan_result** : 예측 모델링을 위해 전처리한 merged_df.csv 사용
- 핀다 내 대출신청 횟수(applied_cnt) : is_applied==1 총합
- 최근성(recency) : 최근 접속일로부터 6/1까지의 기간


# 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cls_df = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/train_log_data_1.csv')

In [ ]:
cls_df.drop(['refer_event_2','refer_event_3','refer_event_4','refer_event_5',
             'week_pop','spend_time','stay_time'], axis=1, inplace=True)
cls_df['timestamp'] = pd.to_datetime(cls_df['timestamp'])
cls_df['month'] = cls_df['timestamp'].dt.month

In [ ]:
print('log_data에서 user_id의 개수: ', cls_df['user_id'].nunique())

## 앱 사용빈도

앱 사용 로그 중 OpenApp이 있으나 OpenApp 로그 없이도 앱 사용 이력이 있는 로그가 존재.
\
전처리된 log_data에는 모든 접속 시작 이력이 표시되어 있으므로 이를 사용. (train_log_data_1.csv)

In [ ]:
openapp_cnt = cls_df[cls_df['refer_event_1']=='start'].groupby(['user_id','month']).count()['refer_event_1']
openapp = openapp_cnt.reset_index()
openapp = openapp.rename(columns = {'refer_event_1':'app_count'})
openapp

In [ ]:
# 3, 4, 5월 별 횟수
openapp['app_count_3'] = openapp['app_count']
openapp.loc[openapp['month']!=3, 'app_count_3'] = 0

openapp['app_count_4'] = openapp['app_count']
openapp.loc[openapp['month']!=4, 'app_count_4'] = 0

openapp['app_count_5'] = openapp['app_count']
openapp.loc[openapp['month']!=5, 'app_count_5'] = 0

In [ ]:
openapp = openapp.groupby('user_id').sum().reset_index().drop('month',axis=1)
openapp

## 서비스별 이용 빈도

이벤트 별로 One-hot encoding을 하고, user_id별로 총합을 구하여 서비스별 이용 횟수의 총합을 구함.

### encoding

In [ ]:
#event
cls_df['event'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
event_cat = ohe.fit_transform(cls_df[['event']])

event_cat = pd.DataFrame(event_cat, columns=[col for col in ohe.categories_[0]])

In [ ]:
event_cat[['user_id','month']] = cls_df[['user_id','month']]
event_cat

### 서비스별 이용 빈도 계산

In [ ]:
event_cnt = event_cat.groupby(['user_id','month']).sum()[['GetCreditInfo','SignUp',
                                                'UseLoanManage','UseDSRCalc','UsePrepayCalc']].reset_index()
event_cnt

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['credit_count_3'] = event_cnt['GetCreditInfo']
event_cnt.loc[event_cnt['month']!=3, 'credit_count_3'] = 0

event_cnt['credit_count_4'] = event_cnt['GetCreditInfo']
event_cnt.loc[event_cnt['month']!=4, 'credit_count_4'] = 0

event_cnt['credit_count_5'] = event_cnt['GetCreditInfo']
event_cnt.loc[event_cnt['month']!=5, 'credit_count_5'] = 0

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['loan_count_3'] = event_cnt['UseLoanManage']
event_cnt.loc[event_cnt['month']!=3, 'loan_count_3'] = 0

event_cnt['loan_count_4'] = event_cnt['UseLoanManage']
event_cnt.loc[event_cnt['month']!=4, 'loan_count_4'] = 0

event_cnt['loan_count_5'] = event_cnt['UseLoanManage']
event_cnt.loc[event_cnt['month']!=5, 'loan_count_5'] = 0

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['dsr_count_3'] = event_cnt['UseDSRCalc']
event_cnt.loc[event_cnt['month']!=3, 'dsr_count_3'] = 0

event_cnt['dsr_count_4'] = event_cnt['UseDSRCalc']
event_cnt.loc[event_cnt['month']!=4, 'dsr_count_4'] = 0

event_cnt['dsr_count_5'] = event_cnt['UseDSRCalc']
event_cnt.loc[event_cnt['month']!=5, 'dsr_count_5'] = 0

In [ ]:
# 3, 4, 5월 별 횟수
event_cnt['prepay_count_3'] = event_cnt['UsePrepayCalc']
event_cnt.loc[event_cnt['month']!=3, 'prepay_count_3'] = 0

event_cnt['prepay_count_4'] = event_cnt['UsePrepayCalc']
event_cnt.loc[event_cnt['month']!=4, 'prepay_count_4'] = 0

event_cnt['prepay_count_5'] = event_cnt['UsePrepayCalc']
event_cnt.loc[event_cnt['month']!=5, 'prepay_count_5'] = 0

In [ ]:
event_cnt = event_cnt.groupby('user_id').sum().reset_index().drop('month',axis=1)
event_cnt

## 핀다 내 대출횟수

user_spec과 loan_result를 결합한 테이블(merged_df.csv)에서 is_applied=1인 횟수를 구하여 유저가 핀다 내에서 얼마나 대출을 신청했는지 횟수를 구함.

In [ ]:
# user_spec과 loan_result가 결합된 테이블 불러오기
result = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/merged_df.csv')

In [ ]:
print('user_spec+loan_result에서 user_id의 개수: ', result['user_id'].nunique()) #259313

In [ ]:
applied_cnt = result.groupby(['user_id','month']).sum()['is_applied'].reset_index()

In [ ]:
# 3, 4, 5월 횟수
applied_cnt['applied_count_3'] = applied_cnt['is_applied']
applied_cnt.loc[applied_cnt['month']!=3, 'applied_count_3'] = 0

applied_cnt['applied_count_4'] = applied_cnt['is_applied']
applied_cnt.loc[applied_cnt['month']!=4, 'applied_count_4'] = 0

applied_cnt['applied_count_5'] = applied_cnt['is_applied']
applied_cnt.loc[applied_cnt['month']!=5, 'applied_count_5'] = 0

In [ ]:
applied_cnt = applied_cnt.groupby('user_id').sum().reset_index().drop('month',axis=1)
applied_cnt.rename(columns={'is_applied':'applied_cnt'}, inplace=True)
applied_cnt

## 데이터 합치기

In [ ]:
cls_df = cls_df.drop_duplicates(['user_id'], keep='last')
cls_df

In [ ]:
print('log_data와 user+loan_result에 공통된 user_id:',
    len(sorted(list(set(cls_df['user_id']) & set(applied_cnt['user_id'])))))

In [ ]:
cls_df = pd.merge(cls_df, openapp, how='inner', on='user_id')

In [ ]:
cls_df = pd.merge(cls_df, event_cnt, how='inner', on='user_id')

In [ ]:
cls_df = pd.merge(cls_df, applied_cnt, how='inner', on='user_id')

## recency

6/1 기준 마지막 접속일로부터 며칠이 지났는지.

In [ ]:
from datetime import datetime

cls_df['recency'] = datetime.strptime('2022-06-01', '%Y-%m-%d') - cls_df['timestamp']
cls_df['recency'] = cls_df['recency'].dt.days

cls_df

## 신규유저

주어진 기간(3, 4, 5월) 내 SignUp 이력이 있는 경우.

In [ ]:
cls_df['new'] = 0

cls_df.loc[cls_df['SignUp']>0, 'new'] = 1
cls_df

## 기능별 시계열 변수

In [ ]:
# 회귀계수
b_app_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,8],cls_df.iloc[i,9], cls_df.iloc[i,10]]
  fit_line = np.polyfit(x, y, 1)
  b_app_array.append(fit_line[0])

b_credit_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,16],cls_df.iloc[i,17], cls_df.iloc[i,18]]
  fit_line = np.polyfit(x, y, 1)
  b_credit_array.append(fit_line[0])

b_prepay_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,25],cls_df.iloc[i,26], cls_df.iloc[i,27]]
  fit_line = np.polyfit(x, y, 1)
  b_prepay_array.append(fit_line[0])

b_dsr_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,22],cls_df.iloc[i,23], cls_df.iloc[i,24]]
  fit_line = np.polyfit(x, y, 1)
  b_dsr_array.append(fit_line[0])

b_loan_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,19],cls_df.iloc[i,20], cls_df.iloc[i,21]]
  fit_line = np.polyfit(x, y, 1)
  b_loan_array.append(fit_line[0])

b_applied_array = []
for i in range(len(cls_df)):
  x = [3, 4, 5]
  y = [cls_df.iloc[i,29],cls_df.iloc[i,30], cls_df.iloc[i,31]]
  fit_line = np.polyfit(x, y, 1)
  b_applied_array.append(fit_line[0])

In [ ]:
cls_df['app_variation'] = b_app_array
cls_df['credit_variation'] = b_credit_array
cls_df['prepay_variation'] = b_prepay_array
cls_df['dsr_variation'] = b_dsr_array
cls_df['loan_variation'] = b_loan_array
cls_df['applied_variation'] = b_applied_array

# 군집 분석
**K - means 모델 사용**
- k = 6

In [ ]:
# cls_df = pd.read_csv('/content/drive/Shareddrives/빅콘테스트/데이터/군집화/cluster_df.csv')

In [ ]:
# 변수 선택
df_cluster = cls_df[['app_count','app_variation',
                 'GetCreditInfo', 'credit_variation',
                 'UsePrepayCalc', 'prepay_variation',
                 'dsr_variation', 'UseDSRCalc',
                 'UseLoanManage', 'loan_variation',
                'applied_cnt', 'applied_variation',
                 'recency', 'new']]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#표준화
sc = StandardScaler()
df_scaled = sc.fit_transform(df_cluster)
pd.DataFrame(df_scaled)

In [ ]:
# 클러스터링
kmeans = KMeans(n_clusters=6, random_state=0)
clusters = kmeans.fit(df_scaled)

# 클러스터링 변수
df_cluster['cluster'] = clusters.labels_
df_cluster.head(2)

KMeans 함수에서 random_state = 0으로 지정해 centroid의 초기값을 고정함. centroid 초기값을 고정하여도 군집 계산 과정에서 완벽하게 재현되지 않음. 군집 라벨링 및 군집 별 수치(평균, 중앙값)가 완벽하게 재현되지 않을 수 있지만 각 군집의 특성과 해석은 변함없음.

In [ ]:
# cluster를 기준으로 데이터 개수
df_cluster.groupby('cluster').count()

In [ ]:
# 그룹별 중앙값
df_cluster.groupby('cluster').median()

In [ ]:
# 그룹별 평균값
df_cluster.groupby('cluster').mean()

In [ ]:
# 모델 저장하기
import joblib
joblib.dump(kmeans, '/content/drive/Shareddrives/빅콘테스트/제출파일/Raw 데이터/clustering_model.pkl') # 불러올 때는 kmeans = joblib.load('파일명')

In [ ]:
import plotly.graph_objects as go

ks = range(1,25)
inertias = []

for k in ks:
  model = KMeans(n_clusters=k)
  model.fit(df_scaled)
  inertias.append(model.inertia_)

In [ ]:
fig = go.Figure(data = go.Scatter(x=list(ks), y = list(inertias), mode = 'lines+markers', ))
fig.update_layout(
    autosize=False,
    width=600)
fig.show()

In [ ]:
df_cluster.to_csv('/final_df_cluster.csv', index = False)